<a href="https://colab.research.google.com/github/rushidarge/Music-Generator-Ai/blob/main/8_6_music_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q streamlit
!pip install colab-everything

     |████████████████████████████████| 8.2MB 3.1MB/s 
     |████████████████████████████████| 174kB 41.2MB/s 
     |████████████████████████████████| 4.2MB 31.0MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 112kB 60.5MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 122kB 51.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
     |████████████████████████████████| 133kB 2.2MB/s 
     |████████████████████████████████| 747kB 3.7MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=912780feb8d14236beeccd8baba13b369e851721a058afe516dd49526413b877
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is in

In [1]:
import numpy as np
# from path import Path
from tqdm import tqdm

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, Embedding

from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint

In [2]:
import requests
url = 'https://raw.githubusercontent.com/rushidarge/Music-Generator-Ai/main/data/input.txt'
page = requests.get(url)
notes = page.text

In [3]:
type(notes)

str

In [4]:
n_vocab = len(set(notes))

In [5]:
pitches = sorted(set(item for item in notes))
char_to_int = dict((num,char) for char,num in enumerate(pitches))
int_to_char = dict((char,num) for char,num in enumerate(pitches))

In [6]:
no_inputs = np.random.randint(2000,15000)
no_inputs

10675

In [7]:
sequence_length = 100
inputs = []
outputs = []
for i in tqdm(range(0,no_inputs-sequence_length)):
    temp_input = notes[i:i+sequence_length] 
    temp_out = notes[i+sequence_length]
    inputs.append([char_to_int[note] for note in temp_input])
    outputs.append(char_to_int[temp_out])
n_patterns = len(inputs)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(inputs, (n_patterns, sequence_length, 1))
# normalize input for better accuracy
network_input = network_input/np.float(n_vocab)
# network_output = to_categorical(outputs)

100%|██████████| 10575/10575 [00:00<00:00, 95484.54it/s]


In [9]:
from keras.models import load_model
model = load_model('/content/kag0706new_music-weights-12-1.2942-bigger.hdf5')

In [10]:
pattern = network_input[np.random.randint(0,len(network_input))]
prediction_output = []

# generate 500 notes
for i in tqdm(range(300)):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_char[index]
    prediction_output.append(result)

    pattern = np.append(pattern,index)
    pattern = pattern[1:len(pattern)]
' '.join([str(elem) for elem in prediction_output])

100%|██████████| 300/300 [01:04<00:00,  4.66it/s]


'" A " A , \n \n A C / 4 2 B B , 4 2 A B , 4 2 A B , 4 2 A B , 4 2 C B , 4 2 C B , 4 2 C B , 4 2 C B , 4 2 C B , 4 2 A B , A A A \n \n " A " A , 3 2 2 A A / 4 2 A A A / 4 4 " A / 4 4 2 " A A / 4 2 A A A / 4 4 A A A / 4 4 4 2 3 " A " A , 4 4 2 " A A / 4 2 A A A / 4 4 | / | | | \\ \n | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | \n \\ : | 2 2 2 2 2 3 2 2 2 2 2 2 2 2 C A A B / 4 2 A " A / \n B | / 4 2 A B / 4 2 A B A / 4 4 A B / 4 4 A , / A A , / / A A , \n 4 4 2 C A A / 4 4 A A / 4 4 A B / 4 4 A B / 4 4 A B / 4 4 A B / 4 4 A B / 4 4 A B / 4 4 A c \n 4 4 4 A d \n 4 4 4 A A / 4 4 A | | | | |'

In [ ]:
%%writefile app.py
import streamlit as st
from tqdm import tqdm
import numpy as np

from keras.models import load_model
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation, Embedding

from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint

import requests

st.write("""
         # Music Generator Ai
         """
         )
st.write("This app gives you musical notes that are generated by Ai")
# file = st.file_uploader("Please upload an image file", type=["jpg", "png"])	
# st.button('Predict')

# load weights into new model
model = load_model("/content/kag0706new_music-weights-12-1.2942-bigger.hdf5",compile=False)

import requests
url = 'https://raw.githubusercontent.com/rushidarge/Music-Generator-Ai/main/data/input.txt'
page = requests.get(url)
notes = page.text

n_vocab = len(set(notes))
pitches = sorted(set(item for item in notes))
char_to_int = dict((num,char) for char,num in enumerate(pitches))
int_to_char = dict((char,num) for char,num in enumerate(pitches))
no_inputs = np.random.randint(2000,15000)

sequence_length = 100
inputs = []
outputs = []
for i in tqdm(range(0,no_inputs-sequence_length)):
    temp_input = notes[i:i+sequence_length] 
    temp_out = notes[i+sequence_length]
    inputs.append([char_to_int[note] for note in temp_input])
    outputs.append(char_to_int[temp_out])
n_patterns = len(inputs)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(inputs, (n_patterns, sequence_length, 1))
# normalize input for better accuracy
network_input = network_input/np.float(n_vocab)
# network_output = to_categorical(outputs)

def prediction_out():
    pattern = network_input[np.random.randint(0,len(network_input))]
    prediction_output = []

    # generate 500 notes
    for i in tqdm(range(50)):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_char[index]
        prediction_output.append(result)

        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]
    return ' '.join([str(elem) for elem in prediction_output])
# ' '.join([str(elem) for elem in prediction_output])

if st.button('Predict'):
    result = prediction_out()
    st.write('Your abc music notations \n')
    st.write(result)

In [26]:
from colab_everything import ColabStreamlit
ColabStreamlit('/content/app.py')

Web App can be accessed on: http://641becc76d87.ngrok.io

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:9999
  External URL: http://35.232.255.45:9999



KeyboardInterrupt: ignored